In [ ]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np


In [10]:
# Cost matrix data
cost_data = {
    'Unnamed: 0': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'],
    'Hub1': [28, 15, 35, 30, 10, 13, 34, 33],
    'Hub2': [12, 17, 25, 38, 24, 39, 15, 19],
    'Hub3': [21, 33, 10, 33, 11, 40, 22, 39],
    'Hub4': [13, 12, 32, 10, 18, 38, 36, 12],
    'Hub5': [31, 12, 11, 30, 38, 38, 17, 24]
}
cost_data

{'Unnamed: 0': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'],
 'Hub1': [28, 15, 35, 30, 10, 13, 34, 33],
 'Hub2': [12, 17, 25, 38, 24, 39, 15, 19],
 'Hub3': [21, 33, 10, 33, 11, 40, 22, 39],
 'Hub4': [13, 12, 32, 10, 18, 38, 36, 12],
 'Hub5': [31, 12, 11, 30, 38, 38, 17, 24]}

In [11]:
# Load the data into a DataFrame
cost_df = pd.read_excel('cost_matrix_multi_hub.xlsx')
cost_df.set_index('Unnamed: 0', inplace=True)

# Transform the DataFrame into a dictionary for combined costs c(i, k, m, j)
cost = {}
for i in cost_df.index:
    for j in cost_df.index:
        if i != j:
            for k in cost_df.columns:
                for m in cost_df.columns:
                    if k != m:
                        cost[(i, k, m, j)] = cost_df.loc[i, k] + cost_df.loc[j, m]
                        
cost

{('A', 'Hub1', 'Hub2', 'B'): 45,
 ('A', 'Hub1', 'Hub3', 'B'): 61,
 ('A', 'Hub1', 'Hub4', 'B'): 40,
 ('A', 'Hub1', 'Hub5', 'B'): 40,
 ('A', 'Hub2', 'Hub1', 'B'): 27,
 ('A', 'Hub2', 'Hub3', 'B'): 45,
 ('A', 'Hub2', 'Hub4', 'B'): 24,
 ('A', 'Hub2', 'Hub5', 'B'): 24,
 ('A', 'Hub3', 'Hub1', 'B'): 36,
 ('A', 'Hub3', 'Hub2', 'B'): 38,
 ('A', 'Hub3', 'Hub4', 'B'): 33,
 ('A', 'Hub3', 'Hub5', 'B'): 33,
 ('A', 'Hub4', 'Hub1', 'B'): 28,
 ('A', 'Hub4', 'Hub2', 'B'): 30,
 ('A', 'Hub4', 'Hub3', 'B'): 46,
 ('A', 'Hub4', 'Hub5', 'B'): 25,
 ('A', 'Hub5', 'Hub1', 'B'): 46,
 ('A', 'Hub5', 'Hub2', 'B'): 48,
 ('A', 'Hub5', 'Hub3', 'B'): 64,
 ('A', 'Hub5', 'Hub4', 'B'): 43,
 ('A', 'Hub1', 'Hub2', 'C'): 53,
 ('A', 'Hub1', 'Hub3', 'C'): 38,
 ('A', 'Hub1', 'Hub4', 'C'): 60,
 ('A', 'Hub1', 'Hub5', 'C'): 39,
 ('A', 'Hub2', 'Hub1', 'C'): 47,
 ('A', 'Hub2', 'Hub3', 'C'): 22,
 ('A', 'Hub2', 'Hub4', 'C'): 44,
 ('A', 'Hub2', 'Hub5', 'C'): 23,
 ('A', 'Hub3', 'Hub1', 'C'): 56,
 ('A', 'Hub3', 'Hub2', 'C'): 46,
 ('A', 'Hu

In [9]:
# Initialize the model
model = pyo.ConcreteModel()

# Define sets
nodes = list(cost_df.index)
hubs = list(cost_df.columns)
pairs = [(i, j) for i in nodes for j in nodes if i != j]
hub_pairs = [(k, m) for k in hubs for m in hubs if k != m]

# Number of hubs to be used
p = 3

# Decision Variables
model.x = pyo.Var(pairs, hub_pairs, within=pyo.Binary)
model.y = pyo.Var(hubs, within=pyo.Binary)

# Objective function: Minimize total cost
def objective_rule(model):
    return sum(cost[(i, k, m, j)] * model.x[(i, j), (k, m)]
               for (i, j) in pairs for (k, m) in hub_pairs)

model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
# Total hubs constraint
def total_hubs_constraint(model):
    return sum(model.y[k] for k in hubs) == p

model.total_hubs_constraint = pyo.Constraint(rule=total_hubs_constraint)

# Allocation constraint
def allocation_constraint(model, i, j):
    return sum(model.x[(i, j), (k, m)] for (k, m) in hub_pairs) == 1

model.allocation_constraint = pyo.Constraint(pairs, rule=allocation_constraint)

# # Flow constraint
# def flow_constraint(model, i, j, k):
#     return sum(model.x[(i, j), (k, m)] + model.x[(i, j), (m, k)] for m in hubs if k != m) <= model.y[k]

# model.flow_constraint = pyo.Constraint(pairs, hubs, rule=flow_constraint)

# First flow constraint
def flow_constraint_a(model, i, j, k):
    return sum(model.x[(i, j), (k, m)] for m in hubs if k != m) <= model.y[k]

model.flow_constraint_a = pyo.Constraint(pairs, hubs, rule=flow_constraint_a)

# Second flow constraint 
def flow_constraint_b(model, i, j, m):
    return sum(model.x[(i, j), (k, m)] for k in hubs if m != k) <= model.y[m]

model.flow_constraint_b = pyo.Constraint(pairs, hubs, rule=flow_constraint_b)

# Non-negativity and binary constraints are implicit in the variable definitions

# Solve the model
opt = pyo.SolverFactory('cbc', executable='bin\\cbc.exe')
results = opt.solve(model)
# model.pprint()

# Extract the objective function value
objective_value = pyo.value(model.obj)
print("\nObjective Function Value:", objective_value)

# Extract results
allocation_plan = pd.DataFrame(
    [(i, j, k, m, pyo.value(model.x[(i, j), (k, m)])) for (i, j) in pairs for (k, m) in hub_pairs if pyo.value(model.x[(i, j), (k, m)]) > 0],
    columns=['Origin', 'Destination', 'First Hub', 'Second Hub', 'Allocation']
)

print("\nAllocation Plan Table")
print(allocation_plan)


Objective Function Value: 1500.0

Allocation Plan Table
   Origin Destination First Hub Second Hub  Allocation
0       A           B      Hub4       Hub5         1.0
1       A           C      Hub4       Hub5         1.0
2       A           D      Hub1       Hub4         1.0
3       A           E      Hub4       Hub1         1.0
4       A           F      Hub4       Hub1         1.0
5       A           G      Hub4       Hub5         1.0
6       A           H      Hub4       Hub5         1.0
7       B           A      Hub5       Hub4         1.0
8       B           C      Hub4       Hub5         1.0
9       B           D      Hub5       Hub4         1.0
10      B           E      Hub4       Hub1         1.0
11      B           F      Hub4       Hub1         1.0
12      B           G      Hub4       Hub5         1.0
13      B           H      Hub5       Hub4         1.0
14      C           A      Hub5       Hub4         1.0
15      C           B      Hub5       Hub4         1.0
16      

In [49]:
# create allocation_plan['Hubs'] column by concatenating 'First Hub' and 'Second Hub' columns in a list value
allocation_plan['Hubs'] = allocation_plan[['First Hub', 'Second Hub']].apply(list, axis=1)
allocation_plan = allocation_plan.drop(['First Hub', 'Second Hub', 'Allocation'], axis=1)


In [50]:
# create a origin - destination matrix containing Hubs column as the value of allocation_plan. so origin will be rows and destinations will be columns
allocation_plan = allocation_plan.pivot_table(index='Origin', columns='Destination', values='Hubs', aggfunc='sum')
allocation_plan

Destination,A,B,C,D,E,F,G,H
Origin,,,,,,,,
A,NaN,"[Hub4, Hub5]","[Hub4, Hub5]","[Hub1, Hub4]","[Hub4, Hub1]","[Hub4, Hub1]","[Hub4, Hub5]","[Hub4, Hub5]"
B,"[Hub5, Hub4]",NaN,"[Hub4, Hub5]","[Hub5, Hub4]","[Hub4, Hub1]","[Hub5, Hub1]","[Hub4, Hub5]","[Hub5, Hub4]"
C,"[Hub5, Hub4]","[Hub5, Hub4]",NaN,"[Hub5, Hub4]","[Hub5, Hub1]","[Hub5, Hub1]","[Hub5, Hub1]","[Hub5, Hub4]"
D,"[Hub4, Hub1]","[Hub4, Hub5]","[Hub4, Hub5]",NaN,"[Hub4, Hub1]","[Hub4, Hub1]","[Hub4, Hub5]","[Hub4, Hub5]"
E,"[Hub1, Hub4]","[Hub1, Hub4]","[Hub1, Hub5]","[Hub1, Hub4]",NaN,"[Hub4, Hub1]","[Hub1, Hub5]","[Hub1, Hub4]"
F,"[Hub1, Hub4]","[Hub1, Hub5]","[Hub1, Hub5]","[Hub1, Hub4]","[Hub1, Hub4]",NaN,"[Hub1, Hub5]","[Hub1, Hub4]"
G,"[Hub5, Hub4]","[Hub5, Hub4]","[Hub1, Hub5]","[Hub5, Hub4]","[Hub5, Hub1]","[Hub5, Hub1]",NaN,"[Hub5, Hub4]"
H,"[Hub5, Hub4]","[Hub4, Hub5]","[Hub4, Hub5]","[Hub5, Hub4]","[Hub4, Hub1]","[Hub4, Hub1]","[Hub4, Hub5]",NaN
